# class2

In [3]:
from numpy import exp, array, random, dot


class NeuralNetwork():
    def __init__(self):
        # Seed the random number generator, so it generates the same numbers
        # every time the program runs.
        random.seed(1)

        # We model a single neuron, with 3 input connections and 1 output connection.
        # We assign random weights to a 3 x 1 matrix, with values in the range -1 to 1
        # and mean 0.
        self.synaptic_weights = 2 * random.random((3, 1)) - 1

    # The Sigmoid function, which describes an S shaped curve.
    # We pass the weighted sum of the inputs through this function to
    # normalise them between 0 and 1.
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # The derivative of the Sigmoid function.
    # This is the gradient of the Sigmoid curve.
    # It indicates how confident we are about the existing weight.
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # We train the neural network through a process of trial and error.
    # Adjusting the synaptic weights each time.
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            # Pass the training set through our neural network (a single neuron).
            output = self.think(training_set_inputs)

            # Calculate the error (The difference between the desired output
            # and the predicted output).
            error = training_set_outputs - output

            # Multiply the error by the input and again by the gradient of the Sigmoid curve.
            # This means less confident weights are adjusted more.
            # This means inputs, which are zero, do not cause changes to the weights.
            adjustment = dot(training_set_inputs.T, error * self.__sigmoid_derivative(output))

            # Adjust the weights.
            self.synaptic_weights += adjustment

    # The neural network thinks.
    def think(self, inputs):
        # Pass inputs through our neural network (our single neuron).
        return self.__sigmoid(dot(inputs, self.synaptic_weights))


if __name__ == "__main__":

    #Intialise a single neuron neural network.
    neural_network = NeuralNetwork()

    print ("Random starting synaptic weights: ")
    print (neural_network.synaptic_weights)
    # The training set. We have 4 examples, each consisting of 3 input values
    # and 1 output value.
    training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
    training_set_outputs = array([[0, 1, 1, 0]]).T #轉至？ 試驗OK

    # Train the neural network using a training set.
    # Do it 10,000 times and make small adjustments each time.
    neural_network.train(training_set_inputs, training_set_outputs, 10000)

    print ("New synaptic weights after training: ")
    print (neural_network.synaptic_weights)

    # Test the neural network with a new situation.
    print ("Considering new situation [1, 0, 0] -> ?: ")
    print (neural_network.think(array([1, 0, 0])))

Random starting synaptic weights: 
[[-0.16595599]
 [ 0.44064899]
 [-0.99977125]]
New synaptic weights after training: 
[[ 9.67299303]
 [-0.2078435 ]
 [-4.62963669]]
Considering new situation [1, 0, 0] -> ?: 
[ 0.99993704]


In [7]:
x = array([[0,1,1,0]]).T

In [8]:
x

array([[0],
       [1],
       [1],
       [0]])

In [6]:
x.T

array([[0],
       [1],
       [1],
       [0]])

# class3

In [3]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb

# IMDB Dataset loading
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000,
                                valid_portion=0.1)
trainX, trainY = train
testX, testY = test

# Data preprocessing
# Sequence padding
trainX = pad_sequences(trainX, maxlen=100, value=0.)
testX = pad_sequences(testX, maxlen=100, value=0.)
# Converting labels to binary vectors
trainY = to_categorical(trainY, nb_classes=2)
testY = to_categorical(testY, nb_classes=2)

# Network building
net = tflearn.input_data([None, 100])
net = tflearn.embedding(net, input_dim=10000, output_dim=128)
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')

# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(trainX, trainY, validation_set=(testX, testY), show_metric=True,
          batch_size=32)

Training Step: 7039  | total loss: 0.06181 | time: 132.359s
| Adam | epoch: 010 | loss: 0.06181 - acc: 0.9874 -- iter: 22496/22500
Training Step: 7040  | total loss: 0.05772 | time: 137.216s
| Adam | epoch: 010 | loss: 0.05772 - acc: 0.9887 | val_loss: 0.72783 - val_acc: 0.8080 -- iter: 22500/22500
--


# Class4 簡單訓練

In [4]:
import numpy as np

In [15]:
# Step 1 Collect Data
x = np.array([[0,0,1],
            [0,1,1],
            [1,0,1],
            [1,1,1]])
                
y = np.array([[0],
            [1],
            [1],
            [0]])

#Step 2 build model

num_epochs = 60000

#initialize weights
syn0 = 2*np.random.random((3,4)) - 1
syn1 = 2*np.random.random((4,1)) - 1


def nonlin(x,deriv=False):
    if(deriv==True):
        return x*(1-x)

    return 1/(1+np.exp(-x))

#Step 3 Train Model

for j in range(num_epochs):
    #feed forward through layers 0,1, and 2
    k0 = x
    k1 = nonlin(np.dot(k0, syn0))#變成1*4的矩陣,[1,2,3,4]
    k2 = nonlin(np.dot(k1, syn1))#變成4*1
    
    #how much did we miss the target value?
    k2_error = y - k2
    
    if (j% 10000) == 0:
        print ("Error:" + str(np.mean(np.abs(k2_error))))
    
    #in what direction is the target value?
    k2_delta = k2_error*nonlin(k2, deriv=True)
    
    #how much did each k1 value contribute to k2 error
    k1_error = k2_delta.dot(syn1.T)
    
    k1_delta= k1_error * nonlin(k1,deriv=True)
    
    syn1 += k1.T.dot(k2_delta)
    syn0 += k0.T.dot(k1_delta)

Error:0.502579215892
Error:0.0118721020065
Error:0.0081187752366
Error:0.00653891594771
Error:0.00561784394993
Error:0.00499748709862


In [18]:
np.dot(x, syn0)

array([[  8.20838376,   0.07757927,   3.13425584,  -0.44752369],
       [  2.73946091,   0.96050992,  -4.09881663,   0.2396529 ],
       [  2.75633627,   0.54408496,  -4.11596433,   0.51557552],
       [ -2.71258659,   1.42701561, -11.3490368 ,   1.20275211]])

In [19]:
k2

array([[ 0.00437122],
       [ 0.99563975],
       [ 0.99563644],
       [ 0.00507964]])

In [20]:
k1

array([[  9.99727712e-01,   5.19384657e-01,   9.58283770e-01,
          3.89949710e-01],
       [  9.39315235e-01,   7.23223400e-01,   1.63215198e-02,
          5.59628245e-01],
       [  9.40270074e-01,   6.32762051e-01,   1.60484815e-02,
          6.26112484e-01],
       [  6.22348608e-02,   8.06435701e-01,   1.17807646e-05,
          7.69014007e-01]])

In [21]:
k0

array([[0, 0, 1],
       [0, 1, 1],
       [1, 0, 1],
       [1, 1, 1]])

In [31]:
import sys
import tensorflow as tf
from termcolor import colored
print(colored('Python Version: %s' % sys.version.split()[0], 'blue'))
print(colored('TensorFlow Ver: %s' % tf.__version__, 'yellow'))

Python Version: 3.5.4
TensorFlow Ver: 1.4.1


# Class5 資料處理

In [32]:
import pandas as pd

In [33]:
df = pd.read_csv("database.csv")

In [36]:
df.Magnitude.head(10)

0    6.0
1    5.8
2    6.2
3    5.8
4    5.8
5    6.7
6    5.9
7    6.0
8    6.0
9    5.8
Name: Magnitude, dtype: float64

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23412 entries, 0 to 23411
Data columns (total 21 columns):
Date                          23412 non-null object
Time                          23412 non-null object
Latitude                      23412 non-null float64
Longitude                     23412 non-null float64
Type                          23412 non-null object
Depth                         23412 non-null float64
Depth Error                   4461 non-null float64
Depth Seismic Stations        7097 non-null float64
Magnitude                     23412 non-null float64
Magnitude Type                23409 non-null object
Magnitude Error               327 non-null float64
Magnitude Seismic Stations    2564 non-null float64
Azimuthal Gap                 7299 non-null float64
Horizontal Distance           1604 non-null float64
Horizontal Error              1156 non-null float64
Root Mean Square              17352 non-null float64
ID                            23412 non-null object
Sou

In [40]:
df.columns

Index(['Date', 'Time', 'Latitude', 'Longitude', 'Type', 'Depth', 'Depth Error',
       'Depth Seismic Stations', 'Magnitude', 'Magnitude Type',
       'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap',
       'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'ID',
       'Source', 'Location Source', 'Magnitude Source', 'Status'],
      dtype='object')

In [43]:
df[["Depth","Root Mean Square","Magnitude"]]

,Depth,Root Mean Square,Magnitude
0,131.60,NaN,6.0
1,80.00,NaN,5.8
2,20.00,NaN,6.2
3,15.00,NaN,5.8
4,15.00,NaN,5.8
5,35.00,NaN,6.7
6,20.00,NaN,5.9
7,35.00,NaN,6.0
8,95.00,NaN,6.0
9,565.00,NaN,5.8


# Class6  Make an Image Classifier

https://github.com/llSourcell/how_to_make_an_image_classifier/blob/master/demo.ipynb

In [44]:
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ImportError: No module named keras


In [46]:
from keras.models import Sequential

Using TensorFlow backend.


In [47]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

In [48]:
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

In [50]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [51]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

/Users/benwang/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,...)`
  from ipykernel import kernelapp as app
/Users/benwang/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/Users/benwang/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


In [52]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [57]:
nb_epoch = 30
nb_train_samples = 200
nb_validation_samples = 50
model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)


/Users/benwang/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:9: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/benwang/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=12, validation_steps=50, epochs=30, validation_data=<keras.pre...)`


Epoch 1/30
12/12 [==============================] - 34s 3s/step - loss: 0.5359 - acc: 0.7240 - val_loss: 0.5060 - val_acc: 0.7469
Epoch 2/30
12/12 [==============================] - 33s 3s/step - loss: 0.5221 - acc: 0.7396 - val_loss: 0.5600 - val_acc: 0.7419
Epoch 3/30
12/12 [==============================] - 33s 3s/step - loss: 0.5114 - acc: 0.7292 - val_loss: 0.4532 - val_acc: 0.7987
Epoch 4/30
12/12 [==============================] - 33s 3s/step - loss: 0.5126 - acc: 0.7656 - val_loss: 0.5083 - val_acc: 0.7532
Epoch 5/30
12/12 [==============================] - 34s 3s/step - loss: 0.5620 - acc: 0.7396 - val_loss: 0.5044 - val_acc: 0.7619
Epoch 6/30
12/12 [==============================] - 33s 3s/step - loss: 0.5042 - acc: 0.7917 - val_loss: 0.5810 - val_acc: 0.7144
Epoch 7/30
12/12 [==============================] - 32s 3s/step - loss: 0.5822 - acc: 0.6771 - val_loss: 0.5225 - val_acc: 0.7069
Epoch 8/30
12/12 [==============================] - 33s 3s/step - loss: 0.5271 - acc: 0.72

In [60]:
model.save_weights('models/basic_cnn_20_epochs.h5')

OSError: Unable to create file (unable to open file: name = 'models/basic_cnn_20_epochs.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)

# Class7 Predict Stock Prices
https://github.com/llSourcell/How-to-Predict-Stock-Prices-Easily-Demo/blob/master/stockdemo.ipynb

In [ ]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import lstm, time #helper libraries

In [ ]:
#Step 1 Load Data
X_train, y_train, X_test, y_test = lstm.load_data('sp500.csv', 50, True)

In [ ]:
#Step 2 Build Model
model = Sequential()

model.add(LSTM(
    input_dim=1,
    output_dim=50,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='rmsprop')
print 'compilation time : ', time.time() - start

#Step 3 Train the model
model.fit(
    X_train,
    y_train,
    batch_size=512,
    nb_epoch=1,
    validation_split=0.05)

#Step 4 - Plot the predictions!
predictions = lstm.predict_sequences_multiple(model, X_test, 50, 50)
lstm.plot_results_multiple(predictions, y_test, 50)